[Reference](https://www.dataquest.io/blog/python-pandas-databases/) <br>
SQLite is a databse engine that makes it simple to store and work with relational data. Python has a library to access SQLite databases called sqlite3 has been included with Python since version `2.5`. <br>

In [1]:
import sqlite3
import pandas as pd
from pandas import DataFrame

# flights.db contains three tables airports, airlines, and routes
conn = sqlite3.connect("flights.db") # create a connection object to a local db
csor = conn.cursor() # a Cursor object allows us to execute SQL queries against a database

# to fetch the first 5 rows from the airlines table
csor.execute("select * from airlines limit 5;") # execute a query using the cursor object's method `execute`.

In [12]:
results = DataFrame(csor.fetchall()) # to assign the result of the query to a variable, use fetchall() to fetch the results.
results # note that the original result is a list of tuples, so we convert the reslut to DataFrame.

,0,1,2,3,4,5,6,7,8,9
0,0,1,Private flight,\N,-,None,None,None,Y,None
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N,None
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y,None
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N,None
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N,None


In [13]:
# much better to create a DataFrame and automatically read the names of the table hearders.
df = pd.read_sql_query("select * from airlines limit 5;", conn)
df

,index,id,name,alias,iata,icao,callsign,country,active,airplanes
0,0,1,Private flight,\N,-,None,None,None,Y,None
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N,None
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y,None
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N,None
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N,None


In [14]:
# Good practice to close connection and cursor objects that are open.
csor.close()
conn.close()

# Modifying database rows
We can use the `sqlite3` package to modify a SQLite database by inserting, updating, or deleting rows.

## Inserting rows with Python

### [1] Hardcoding value into the database

In [2]:
import sqlite3
import pandas as pd
from pandas import DataFrame

# flights.db contains three tables airports, airlines, and routes
conn = sqlite3.connect("flights.db") # create a connection object to a local db

df = pd.read_sql_query("select * from airlines", conn)
conn.close()
df.tail(10)

# The practice below may add a column "airplanes" to the original db.
# If ths is the case, you need to rename the old one and copy the original column to the new one.

,index,id,name,alias,iata,icao,callsign,country,active,airplanes
6041,6041,19815,Regionalia Venezuela,Regionalia Venezuela,9X,9XX,None,Venezuela,Y,None
6042,6042,19827,Regionalia Chile,Regionalia Chile,9J,CR1,None,Chile,Y,None
6043,6043,19828,Vuela Cuba,Vuela Cuba,6C,6CC,None,Cuba,Y,None
6044,6044,19830,All Australia,All Australia,88,8K8,None,Australia,Y,None
6045,6045,19831,Fly Europa,None,ER,RWW,None,Spain,Y,None
6046,6046,19834,FlyPortugal,None,PO,FPT,FlyPortugal,Portugal,Y,None
6047,6047,19845,FTI Fluggesellschaft,None,None,FTI,None,Germany,N,None
6048,6048,19846,Test flight,,,None,None,None,Y,None
6049,6048,19846,Test flight,,,None,None,None,Y,None
6050,6048,19846,Test flight,,,None,None,None,Y,None


#### Displaying column datatype

In [9]:
import sqlite3
conn = sqlite3.connect('flights.db')
csor = conn.cursor()
msg = csor.execute("""
    PRAGMA table_info (airlines) 
""").fetchall()
print(msg)

conn.close()

[(0, 'index', 'INTEGER', 0, None, 0), (1, 'id', 'TEXT', 0, None, 0), (2, 'name', 'TEXT', 0, None, 0), (3, 'alias', 'TEXT', 0, None, 0), (4, 'iata', 'TEXT', 0, None, 0), (5, 'icao', 'TEXT', 0, None, 0), (6, 'callsign', 'TEXT', 0, None, 0), (7, 'country', 'TEXT', 0, None, 0), (8, 'active', 'TEXT', 0, None, 0), (9, 'airplanes', 'integer', 0, None, 0)]


#### Keeping part of columns
Run this subsection, only when you encouter the problem where airlines table has 10 rather than the original 9 columns.

In [24]:
# In sqlite, you cannot drop columns as you do in other sql languages.
# You need to rename the old db, copy the desired column to a new one, and in the end drop the old db.

import sqlite3
conn = sqlite3.connect('flights.db')
csor = conn.cursor()
csor.execute("ALTER TABLE airlines RENAME TO _airlines_old;")
csor.execute("""
    CREATE TABLE airlines
    (index INTEGER,
     id INTEGER,
     name TEXT,
     alias TEXT,
     iata TEXT,
     icao TEXT,
     callsign TEXT,
     country TEXT,
     active TEXT
    );""")
csor.execute("""
    INSERT INTO airlines (index, id, name, alias, iata, icao, callsign, country, active)
        SELECT index, id, name, alias, iata, icao, callsign, country, active FROM _airlines_old;
""")
conn.commit()

OperationalError: no such table: airlines

In [16]:
conn = sqlite3.connect("flights.db")
csor = conn.cursor()
csor.execute("""
insert into airlines 
values (6048, 19846, 'Test flight', '', '', null, null, null, 'Y')
""")

If you try to query the table now, you won't see the new row yet.
SQLite doesn't write to the database until you commit a transaction, which consists of one or more queries. A transaction won't commit
until all the queries succeed to avoid inconsistency among tables.

Sidenote: why do I get a database lock message? [reference](https://www.dataquest.io/blog/python-pandas-databases/#insertingrowswithpython)

In [3]:
conn.commit()
df = pd.read_sql_query("select * from airlines", conn)
df.tail(10)

# 
conn = sqlite3.connect('flights.db')
df = pd.read_sql_query()

,index,id,name,alias,iata,icao,callsign,country,active
6041,6041,19815,Regionalia Venezuela,Regionalia Venezuela,9X,9XX,None,Venezuela,Y
6042,6042,19827,Regionalia Chile,Regionalia Chile,9J,CR1,None,Chile,Y
6043,6043,19828,Vuela Cuba,Vuela Cuba,6C,6CC,None,Cuba,Y
6044,6044,19830,All Australia,All Australia,88,8K8,None,Australia,Y
6045,6045,19831,Fly Europa,None,ER,RWW,None,Spain,Y
6046,6046,19834,FlyPortugal,None,PO,FPT,FlyPortugal,Portugal,Y
6047,6047,19845,FTI Fluggesellschaft,None,None,FTI,None,Germany,N
6048,6048,19846,Test flight,,,None,None,None,Y
6049,6048,19846,Test flight,,,None,None,None,Y
6050,6048,19846,Test flight,,,None,None,None,Y


In [17]:
csor.close()
conn.close()

### [2] Insert values with string formatting

In [30]:
conn = sqlite3.connect("flights.db")
csor = conn.cursor()
values = ('Test Flight', 'Y')
csor.execute("insert into airlines values (6049, 19847, ?, '', '', null, null, null, ?)", values)
conn.commit()
df = pd.read_sql_query("select * from airlines", conn)
csor.close()
conn.close()
df.tail(10)

,index,id,name,alias,iata,icao,callsign,country,active
6047,6047,19845,FTI Fluggesellschaft,None,None,FTI,None,Germany,N
6048,6048,19846,Test flight,,,None,None,None,Y
6049,6048,19846,Test flight,,,None,None,None,Y
6050,6048,19846,Test flight,,,None,None,None,Y
6051,6049,19847,Test Flight,,,None,None,None,Y
6052,6049,19847,Test Flight,,,None,None,None,Y
6053,6049,19847,Test Flight,,,None,None,None,Y
6054,6049,19847,Test Flight,,,None,None,None,Y
6055,6049,19847,Test Flight,,,None,None,None,Y
6056,6049,19847,Test Flight,,,None,None,None,Y


## Deleting rows

In [39]:
import sqlite3
conn = sqlite3.connect("flights.db")
csor = conn.cursor()
values = (19847,) # comma is not optional.
csor.execute("delete from airlines where id=?", values)
df = pd.read_sql_query("select * from airlines", conn)
conn.commit()
csor.close()
conn.close()
df.tail(10)

,index,id,name,alias,iata,icao,callsign,country,active
6031,6031,19805,All Asia,All Asia,L9,AL3,None,China,Y
6032,6032,19806,All Africa,All Africa,9A,99F,None,South Africa,Y
6033,6033,19807,Regionalia México,Regionalia México,N4,J88,None,Mexico,Y
6034,6034,19808,All Europe,All Europe,N9,N99,None,United Kingdom,Y
6035,6035,19809,All Spain,All Spain,N7,N77,None,Spain,Y
6036,6036,19810,Regional Air Iceland,Regional Air Iceland,9N,N78,None,Iceland,Y
6037,6037,19811,British Air Ferries,None,??,??!,None,United Kingdom,N
6038,6038,19812,Voestar,Voestar Brasil,8K,K88,None,Brazil,Y
6039,6039,19813,All Colombia,All Colombia,7O,7KK,None,Colombia,Y
6040,6040,19814,Regionalia Uruguay,Regionalia Uruguay,2X,2K2,None,Uruguay,Y


## Creating tables

In [11]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("flights.db")
csor = conn.cursor()

csor.execute("""
create table daily_flights
(id integer, departure date, arrival date, number text, route_id integer)
""")
conn.commit()

csor.execute("""
insert into daily_flights
values (1, '2016-09-28 0:00', '2016-09-28 12:00', 'T1', 1)
""")
conn.commit()

df = pd.read_sql_query("select * from daily_flights", conn)
csor.close()
conn.close()
df.tail(10)

,id,departure,arrival,number,route_id
0,1,2016-09-28 0:00,2016-09-28 12:00,T1,1


## Removing tables

In [10]:
import sqlite3
conn = sqlite3.connect("flights.db")
csor = conn.cursor()
csor.execute("drop table daily_flights")
conn.commit()
csor.close()
conn.close()

## Creating tables with pandas

In [2]:
from datetime import datetime
import pandas as pd
import sqlite3
conn = sqlite3.connect("flights.db")
df = pd.DataFrame(
    [[1, datetime(2016, 9, 29, 0, 0) , datetime(2016, 9, 29, 12, 0), 'T1', 1]], 
    columns=["id", "departure", "arrival", "number", "route_id"]    
)

# to convert df to a table in a database 
df.to_sql("daily_flights", conn, if_exists = "replace")

# to verify that everything worked by querying the database
df = pd.read_sql_query("select * from daily_flights;", conn)
conn.close()

df

,index,id,departure,arrival,number,route_id
0,0,1,2016-09-29 00:00:00,2016-09-29 12:00:00,T1,1


## Adding a column with pandas

In [3]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("flights.db")
csor = conn.cursor()
csor.execute("""
alter table airlines 
add column airplanes integer
""")
conn.commit()
df = pd.read_sql_query("select * from airlines limit 1", conn)
csor.close()
conn.close()
print(df)

OperationalError: duplicate column name: airplanes

In [4]:
csor.close()
conn.close()

In [10]:
# Note that DROP COLUMN doensn't work in sqlite
# Thsu, the operation of this cell fails.

import pandas as pd
import sqlite3

conn = sqlite3.connect("flights.db")
csor = conn.cursor()
csor.execute("alter table airlines drop column airplanes")

csor.close()
conn.close()

OperationalError: near "drop": syntax error

# Mapping airports
[Reference](https://www.dataquest.io/blog/python-pandas-databases/#mappingroutes): Dataquest
—
Working with SQLite Databases using Python and Pandas <br> 

In [14]:
import sqlite3
conn = sqlite3.connect("flights.db") # access to a local db
csor = conn.cursor() # allow us to execute SQL queries against a database

# retrieve the latitude and longitude columns from airports
# , and convert them to floats. Then we call fetchall() to retrieve them
coords = csor.execute("""
    select cast(longitude as float),
    cast(latitude as float)
    from airports;
""").fetchall()

In [15]:
import Basemap
import matplotlib.pyplot as plt

# map setup:
# draw the continets and coastlines 
# that will from the background of our map

m = Basemap(
  projection='merc',
  llcrnrlat=-80,
  urcrnrlat=80,
  llcrnrlon=-180,
  urcrnrlon=180,
  lat_ts=20,
  resolution='c'
)

m.drawcoastlines()
m.drawmapboundary()

# I didn't proceed, because Basemap and Tensorflow conflinct.

ModuleNotFoundError: No module named 'mpl_toolkits'

# Import db from a local file

## [1] Through sqlite3 package

In [23]:
import sqlite3

# data source: 
# https://github.com/jpwhite3/northwind-SQLite3
sqlite_file = r'C:\Users\libin\Desktop\Northwind_large.sqlite' 
conn = sqlite3.connect(sqlite_file)
conn.close()

# Northwind Database
## Show list of tables

In [24]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///Northwind_small.sqlite')
print(engine.table_names())

['Category', 'Customer', 'CustomerCustomerDemo', 'CustomerDemographic', 'Employee', 'EmployeeTerritory', 'Order', 'OrderDetail', 'Product', 'Region', 'Shipper', 'Supplier', 'Territory']


## Import .sqlite files 

In [1]:
# Be sure you have installed ipython-sql. 
# If you do, run the following...
%reload_ext sql

C:\Users\libin\Anaconda3\envs\tfgpu\lib\site-packages\IPython\config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
C:\Users\libin\Anaconda3\envs\tfgpu\lib\site-packages\sql\magic.py:4: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils.traitlets import Bool, Int, Unicode


In [27]:
# source 1: https://github.com/jpwhite3/northwind-SQLite3
# source 2: https://northwinddatabase.codeplex.com/downloads/get/269239
# example: %sql sqlite:///flights.db
%sql sqlite:///Northwind_small.sqlite

'Connected: None@Northwind_small.sqlite'

A link for learning sqlite: https://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html <br>
Drop column in sqlite: https://www.techonthenet.com/sqlite/tables/alter_table.php <br>
Northwind question set No.1: https://www.youtube.com/watch?v=3j9PFyvakOA <br>
SQL project: https://www.youtube.com/channel/UCvIHnJ8croj_v2fx_ZXuTlw <br>
Google Cloud python: https://www.youtube.com/watch?v=chk2rRjSn5o <br>
Google compute engine: https://www.youtube.com/watch?annotation_id=annotation_1708545089&feature=iv&src_vid=LrjpcR-IJwY&v=gxZvofAvgHQ <br>
Create table: https://www.youtube.com/watch?v=NCc5r7Wr7gg <br>
KD projects: https://www.kdnuggets.com/2017/05/data-science-tutorial-series-software-engineers.html <br>
install Northwind into SQL servor 2014: https://www.youtube.com/watch?v=iKVbx5IeUvQ <br>

In [34]:
import sqlite3
conn = sqlite3.connect('Northwind_small.sqlite')
csor = conn.cursor()
df = csor.execute("SELECT * FROM Employee LIMIT 3").fetchall()
print(df)
conn.close()                       

[(1, 'Davolio', 'Nancy', 'Sales Representative', 'Ms.', '1980-12-08', '2024-05-01', '507 - 20th Ave. E. Apt. 2A', 'Seattle', 'North America', '98122', 'USA', '(206) 555-9857', '5467', None, "Education includes a BA in psychology from Colorado State University in 1970.  She also completed 'The Art of the Cold Call.'  Nancy is a member of Toastmasters International.", 2, 'http://accweb/emmployees/davolio.bmp'), (2, 'Fuller', 'Andrew', 'Vice President, Sales', 'Dr.', '1984-02-19', '2024-08-14', '908 W. Capital Way', 'Tacoma', 'North America', '98401', 'USA', '(206) 555-9482', '3457', None, 'Andrew received his BTS commercial in 1974 and a Ph.D. in international marketing from the University of Dallas in 1981.  He is fluent in French and Italian and reads German.  He joined the company as a sales representative, was promoted to sales manager in January 1992 and to vice president of sales in March 1993.  Andrew is a member of the Sales Management Roundtable, the Seattle Chamber of Commerce,